**IMPORTAR LIBRERIAS**

---

In [ ]:
import pandas as pd

In [ ]:
from db_connection import get_connection
from mysql.connector import Error

# Crear conexión a la base de datos
conexion = get_connection()
cursor = conexion.cursor()

In [ ]:
from db_connection import get_engine  

# Crear engine para acceder a la base de datos
engine = get_engine()

---


**FUNCIONES**

---

In [ ]:
# Comprobar si la tabla esta relacionada con otra
def columna_tiene_foreign_key(cursor, tabla, columna):
    cursor.execute(f"""
        SELECT COUNT(*) 
        FROM information_schema.KEY_COLUMN_USAGE 
        WHERE TABLE_NAME = '{tabla}' 
        AND COLUMN_NAME = '{columna}'
        AND TABLE_SCHEMA = DATABASE()
        AND REFERENCED_TABLE_NAME IS NOT NULL;
    """)
    return cursor.fetchone()[0] > 0


In [ ]:
# Obtener el nombre de la clave foránea de la tabla relacionada
def obtener_nombre_clave_foranea(cursor, tabla, columna):
    cursor.execute(f"""
        SELECT CONSTRAINT_NAME 
        FROM information_schema.KEY_COLUMN_USAGE 
        WHERE TABLE_NAME = '{tabla}' 
        AND COLUMN_NAME = '{columna}' 
        AND TABLE_SCHEMA = DATABASE()
        AND REFERENCED_TABLE_NAME IS NOT NULL;
    """)
    result = cursor.fetchone()
    return result[0] if result else None

In [ ]:
# Verificar columnas completamente nulas
def columnas_vacias(columnas, tabla):

    columnas_eliminadas = []

    for col in columnas:
        cursor.execute(f"SELECT COUNT(*) FROM `{tabla}` WHERE `{col}` IS NOT NULL;")
        not_null_count = cursor.fetchone()[0]
        if not_null_count == 0:
            try:
                if columna_tiene_foreign_key(cursor, tabla, col):
                    fk_name = obtener_nombre_clave_foranea(cursor, tabla, col)
                    if fk_name:
                        cursor.execute(f"ALTER TABLE `{tabla}` DROP FOREIGN KEY `{fk_name}`;")
                        print(f"🔓 Foreign key '{fk_name}' eliminada de columna '{col}'")

                cursor.execute(f"ALTER TABLE `{tabla}` DROP COLUMN `{col}`;")
                columnas_eliminadas.append(col)
                print(f"🗑️ Columna eliminada: {col}")


            except Error as e:
                print(f"❌ Error al eliminar columna {col}: {e}")
    return columnas_eliminadas

In [ ]:
# Verificar filas completamente nulas
def filas_completamente_nulas(columnas_eliminadas, columnas, tabla):
    # Eliminar filas completamente nulas (todas las columnas NULL)
    if columnas_eliminadas:
        # actualizar la lista de columnas
        cursor.execute(f"SHOW COLUMNS FROM `{tabla}`;")
        columnas = [col[0] for col in cursor.fetchall()]

    if columnas:
        condiciones = " AND ".join([f"`{col}` IS NULL" for col in columnas])
        delete_query = f"DELETE FROM `{tabla}` WHERE {condiciones};"
        try:
            cursor.execute(delete_query)
            filas_afectadas = cursor.rowcount
            if filas_afectadas > 0:
                print(f"🗑️ Filas eliminadas completamente nulas: {filas_afectadas}")
        except Error as e:
            print(f"❌ Error al eliminar filas nulas: {e}")
    else:
        print("⚠️ La tabla quedó sin columnas, se omite eliminación de filas.")

---

**LIMPIEZA DE DATOS NULOS**

---

In [ ]:
# Obtener todas las tablas
cursor.execute("SHOW TABLES;")
tablas = [t[0] for t in cursor.fetchall()]

In [ ]:
# Ejecutar limpieza en cada tabla
for tabla in tablas:
    print(f"\n🔄 Procesando tabla: {tabla}")
    
    # Obtener nombres de columnas
    cursor.execute(f"SHOW COLUMNS FROM `{tabla}`;")
    columnas = [col[0] for col in cursor.fetchall()]
    
    columnas_eliminadas = columnas_vacias(columnas, tabla)
    
    filas_completamente_nulas(columnas_eliminadas, columnas, tabla)
    


# Confirmar cambios
conexion.commit()
print("\n✅ Limpieza completada.")

In [ ]:
# Cerrar conexión
cursor.close()
conexion.close()